# Todo
1. use [center loss](https://medium.com/mlreview/experiments-with-a-new-loss-term-added-to-the-standard-cross-entropy-85b080c42446)
2. split train & val
3. add [Beam Search](https://machinelearningmastery.com/beam-search-decoder-natural-language-processing/)
4. maybe add [attention](https://towardsdatascience.com/light-on-math-ml-attention-with-keras-dc8dbc1fad39)
5. add [reg](https://machinelearningmastery.com/how-to-reduce-overfitting-with-dropout-regularization-in-keras/)

In [6]:
import warnings
warnings.filterwarnings("ignore")
import src.dataset as ds
import numpy as np
from src.embeddings import extract_embedding_weights
from keras.layers import Embedding, CuDNNLSTM, Bidirectional, Dense, CuDNNGRU
from keras.initializers import Constant
from keras import Sequential
from tensorflow.python.keras.callbacks import TensorBoard
from src.model import Model

from time import time
import tensorflow as tf


In [7]:
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)

In [8]:
X, y, tokenizer = ds.load_tokenized_data()
embedding_matrix = extract_embedding_weights()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 615/615 [00:01<00:00, 362.16it/s]


In [9]:
model = Model(tokenizer, embedding_matrix)
model.train(X,y, epochs=2, batch_size=20, callbacks=[tensorboard])
# model.test()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1, 300)            2251800   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100)               140800    
_________________________________________________________________
dense_1 (Dense)              (None, 7506)              758106    
Total params: 3,150,706
Trainable params: 898,906
Non-trainable params: 2,251,800
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Train on 154596 samples, validate on 38650 samples
Epoch 1/2
154596/154596 [==============================] - 42s 275us/step - loss: 5.1336 - _perplexity: 769.6594 - val_loss: 5.4209 - val__perplexity: 2604.0124
Epoch 2/2
154596/154596 [==============================] -

In [ ]:
model.predict('Deep', 10)

> c:\users\eli\workspace\deep-learning-intro\assignment3\src\model.py(71)predict()
     70             ipdb.set_trace()
---> 71             out_word = ''
     72             for word, index in tokenizer.word_index.items():



ipdb>  proba


array([[2.6724436e-09, 2.8712122e-09, 1.7368726e-05, ..., 1.6459103e-06,
        5.8268895e-04, 2.2899949e-05]], dtype=float32)


ipdb>  proba.shape


(1, 7506)


...

[array([2.6724436e-09, 2.8712122e-09, 1.7368726e-05, ..., 1.6459103e-06,
       5.8268895e-04, 2.2899949e-05], dtype=float32)]


ipdb>  proba


array([[2.6724436e-09, 2.8712122e-09, 1.7368726e-05, ..., 1.6459103e-06,
        5.8268895e-04, 2.2899949e-05]], dtype=float32)


ipdb>  -np.sort(-proba)


array([[3.9417145e-01, 5.5460762e-02, 2.6946710e-02, ..., 2.4793119e-09,
        1.9104485e-09, 1.1865843e-09]], dtype=float32)


In [ ]:
# beam search
def beam_search_decoder(first_word, n_words, beam_width=3):
    tokenizer = self.tokenizer
    model = self.model
    in_text, result = first_word, first_word
    
	sequences = [[list(), 1.0]]
	# walk over each step in sequence
	for row in data:
		all_candidates = list()
		# expand each current candidate
		for i in range(len(sequences)):
			seq, score = sequences[i]
			for j in range(len(row)):
				candidate = [seq + [j], score * -log(row[j])]
				all_candidates.append(candidate)
		# order all candidates by score
		ordered = sorted(all_candidates, key=lambda tup:tup[1])
		# select k best
		sequences = ordered[:k]
	return sequences
 

In [ ]:
def beam_step(beam_sequences_scores):            
    all_candidates = []
    encoded = get_encoded(in_text, tokenizer)    
    for seq, score in beam_sequences_scores: # for each sequence
        seq.extend(encoded) # add words till now
        # predict top B words
        words_probs = model.predict_proba(encoded, verbose=0)
        words_probs_sorted = -np.sort(-words_probs) # sorting in descending order
        top_b_words_probs = words_probs_sorted[:B] # top B words with max probability
        # for each prob in top B words, create a candidate
        for prob in top_b_words_probs: 
            candidate = [seq + [word_token], score + np.log(prob + epsilon())] # todo: word_token 
            all_candidates.append(candidate)
    # take candidates with max score
    beam_sequences_scores = sorted(all_candidates, reverse=True, key=lambda seq, score: score)[:B]
    return beam_sequences_scores

In [ ]:
from keras.backend import epsilon
def predict(self, first_word, n_words, B=3):
        tokenizer = self.tokenizer
        model = self.model
        in_text, result = first_word, first_word
        beam_sequences_scores = [[list(), 0]]
        while len(result) < n_words: 
           
            for _ in range(B): # run till beam_width
                

                
                
        
def get_encoded(text, tokenizer):
    encoded = tokenizer.texts_to_sequences([text])[0]
    encoded = np.array(encoded)
    return encoded

In [ ]:
        
        
        # generate a fixed number of words
        for _ in range(n_words):
            # encode the text as integer
            encoded = tokenizer.texts_to_sequences([in_text])[0]
            encoded = np.array(encoded)
            # predict a word in the vocabulary
            proba = model.predict_proba(encoded, verbose=0)
            import ipdb
            ipdb.set_trace()
            out_word = ''
            for word, index in tokenizer.word_index.items():
                if index == yhat:
                    out_word = word
                    break
            # append to input
            in_text, result = out_word, result + ' ' + out_word
        return result